In [1]:
import time
import read_espn
import pandas as pd
import os

In [3]:
gameids, results = read_espn.get_gameid(2017,4)
print(gameids)
print(results)

['400951678', '400950241', '400951685', '400951712', '400951716', '400951724', '400951727', '400951730', '400951708', '400951720', '400951735', '400951738', '400951742', '400951744', '400951747', '400951752']
{'400951678': 'GB 35, CHI 14', '400950241': 'NO 20, MIA 0', '400951685': 'BUF 23, ATL 17', '400951712': 'CIN 31, CLE 7', '400951716': 'LAR 35, DAL 30', '400951724': 'DET 14, MIN 7', '400951727': 'CAR 33, NE 30', '400951730': 'NYJ 23, JAX 20 (OT)', '400951708': 'PIT 26, BAL 9', '400951720': 'HOU 57, TEN 14', '400951735': 'ARI 18, SF 15 (OT)', '400951738': 'PHI 26, LAC 24', '400951742': 'TB 25, NYG 23', '400951744': 'DEN 16, OAK 10', '400951747': 'SEA 46, IND 18', '400951752': 'KC 29, WSH 20'}


In [4]:
gameids, results = read_espn.get_gameid(2018,4)
print(gameids)
print(results)

['401030839', '401030847', '401030846', '401030845', '401030844', '401030842', '401030843', '401030841', '401030840', '401030930', '401030848', '401030931', '401030932', '401030933', '401030934']
{'401030839': [], '401030847': [], '401030846': [], '401030845': [], '401030844': [], '401030842': [], '401030843': [], '401030841': [], '401030840': [], '401030930': [], '401030848': [], '401030931': [], '401030932': [], '401030933': [], '401030934': []}


In [5]:
def weekly_games(season,week):
    gameids, results = read_espn.get_gameid(season,week)
    data = [ 
        {
            'gameid':gid,
            'season':season,
            'week':week,
            'result':results[gid]
        } for i, gid in enumerate(gameids)
    ]
    gamedata_df = pd.DataFrame(data)
    # Set the gameId as the unique identifier for each row
    gamedata_df['gameid'] = gamedata_df['gameid'].astype(str)
    gamedata_df.set_index('gameid', inplace=True)
    return gamedata_df

In [6]:
weekly_games(2018,4)

,result,season,week
gameid,,,
401030839,[],2018,4
401030847,[],2018,4
401030846,[],2018,4
401030845,[],2018,4
401030844,[],2018,4
401030842,[],2018,4
401030843,[],2018,4
401030841,[],2018,4
401030840,[],2018,4


In [7]:
# Given a dataframe whose index consists of gameids
# Scrape drive and play-by-play data for the games

def scrape_gameset(games_df):
    # List of DFs for each game
    pbp_list = []
    drivelevel_list = []
    game_home = {}
    game_away = {}
    
    for gid in games_df.index:
        try:
            print(gid)
            pbp_df, drives_df = read_espn.get_game_df(gid)
            # Add column for gameid
            pbp_df['gameid'] = gid
            drives_df['gameid'] = gid
            # Extract home/away from game df
            game_home[gid] = pbp_df['home'].values[0]
            game_away[gid] = pbp_df['away'].values[0]
            
            pbp_list.append(pbp_df)
            drivelevel_list.append(drives_df)
            
        except:
            print("Failed to scrape gameid",gid)
            
        time.sleep(0.25)
        
    # Merge DFs together
    allplays_df = pd.concat(pbp_list)
    alldrives_df = pd.concat(drivelevel_list)
    
    # Add home/away coloumns to games_df
    games_df['home'] = pd.Series(game_home)
    games_df['away'] = pd.Series(game_away)
    
    return (games_df, alldrives_df, allplays_df)

In [8]:
read_espn.get_game_df('400951620')

(                downdist                                             detail  \
 0                     []  (15:00 - 1st) R.Bullock kicks 65 yards from CI...   
 1   [1st & 10 at HOU 25]  (15:00 - 1st)  L.Miller right guard to HST 29 ...   
 2    [2nd & 6 at HOU 29]  (14:25 - 1st)  D.Watson pass incomplete short ...   
 3    [3rd & 6 at HOU 29]  (14:19 - 1st)  (Shotgun) D.Watson pass incompl...   
 4    [4th & 6 at HOU 29]  (14:14 - 1st)  S.Lechler punts 58 yards to CIN...   
 0   [1st & 10 at CIN 46]  (13:59 - 1st)  J.Hill right tackle to HST 49 f...   
 1    [2nd & 5 at HOU 49]  (13:24 - 1st)  A.Dalton pass short left to A.G...   
 2   [2nd & 14 at CIN 42]  (13:01 - 1st)  (Shotgun) A.Dalton pass short m...   
 3    [3rd & 8 at CIN 48]  (12:28 - 1st)  (Shotgun) A.Dalton sacked at CI...   
 4   [4th & 12 at CIN 44]  (12:21 - 1st)  (Punt formation) PENALTY on CIN...   
 5   [4th & 17 at CIN 39]  (12:09 - 1st)  K.Huber punts 58 yards to HST 3...   
 0    [1st & 10 at HOU 3]  (11:59 - 1st)

In [9]:
games_df = weekly_games(2017,2)
(games_df, drives_df, plays_df) = scrape_gameset(games_df)

400951620
400951632
400951636
400951639
400951643
400951645
400951630
400951635
400951626
400951669
400951666
400951673
400951674
400951676
400951679
400951681


In [10]:
games_df

,result,season,week,home,away
gameid,,,,,
400951620,"HOU 13, CIN 9",2017,2,HOU,CIN
400951632,"ARI 16, IND 13 (OT)",2017,2,ARI,IND
400951636,"KC 27, PHI 20",2017,2,PHI,KC
400951639,"NE 36, NO 20",2017,2,NE,NO
400951643,"PIT 26, MIN 9",2017,2,MIN,PIT
400951645,"TB 29, CHI 7",2017,2,CHI,TB
400951630,"CAR 9, BUF 3",2017,2,BUF,CAR
400951635,"TEN 37, JAX 16",2017,2,TEN,JAX
400951626,"BAL 24, CLE 10",2017,2,CLE,BAL


In [21]:
# One-time use to get gameids history
weekly_dfs = []
for season in range(2000,2018):
    for week in range(1,18):
        print(season,": week",week)
        df = read_espn.weekly_games(season,week)
        weekly_dfs.append(df)
        time.sleep(0.25)
        
game_history_df = pd.concat(weekly_dfs)
game_history_df

2000 : week 1
2000 : week 2
2000 : week 3
2000 : week 4
2000 : week 5
2000 : week 6
2000 : week 7
2000 : week 8
2000 : week 9
2000 : week 10
2000 : week 11
2000 : week 12
2000 : week 13
2000 : week 14
2000 : week 15
2000 : week 16
2000 : week 17
2001 : week 1
2001 : week 2
2001 : week 3
2001 : week 4
2001 : week 5
2001 : week 6
2001 : week 7
2001 : week 8
2001 : week 9
2001 : week 10
2001 : week 11
2001 : week 12
2001 : week 13
2001 : week 14
2001 : week 15
2001 : week 16
2001 : week 17
2002 : week 1
2002 : week 2
2002 : week 3
2002 : week 4
2002 : week 5
2002 : week 6
2002 : week 7
2002 : week 8
2002 : week 9
2002 : week 10
2002 : week 11
2002 : week 12
2002 : week 13
2002 : week 14
2002 : week 15
2002 : week 16
2002 : week 17
2003 : week 1
2003 : week 2
2003 : week 3
2003 : week 4
2003 : week 5
2003 : week 6
2003 : week 7
2003 : week 8
2003 : week 9
2003 : week 10
2003 : week 11
2003 : week 12
2003 : week 13
2003 : week 14
2003 : week 15
2003 : week 16
2003 : week 17
2004 : week 1
20

,result,season,week
gameid,,,
200903001,"ATL 36, SF 28",2000,1
200903005,"JAX 27, CLE 7",2000,1
200903012,"IND 27, KC 14",2000,1
200903016,"MIN 30, CHI 27",2000,1
200903017,"TB 21, NE 16",2000,1
200903018,"DET 14, NO 10",2000,1
200903019,"NYG 21, ARI 16",2000,1
200903023,"BAL 16, PIT 0",2000,1
200903028,"WSH 20, CAR 17",2000,1


In [22]:
histfile = "/home/welced12/googledrive/nfl_data/devl/espn_gamedata.json"
game_history_df.to_json(histfile)

In [23]:
read_hist = pd.read_json(histfile, convert_axes=False)
read_hist.tail(10)

,result,season,week
400951810,"CIN 20, DEN 17",2017,11
400951811,"LAR 42, SEA 7",2017,15
400951812,"NE 27, PIT 24",2017,15
400951813,"SF 25, TEN 23",2017,15
400951814,"DAL 20, OAK 17",2017,15
400951815,"NE 33, OAK 8",2017,11
400951816,"ATL 24, TB 21",2017,15
400951817,"PHI 37, DAL 9",2017,11
400951818,"ATL 34, SEA 31",2017,11
400981391,"TB 30, MIA 20",2017,11


In [24]:
# Add a week to existing game history
new_week = read_espn.weekly_games(2018,1)
new_week

,result,season,week
gameid,,,
401030710,[],2018,1
401030718,[],2018,1
401030717,[],2018,1
401030716,[],2018,1
401030715,[],2018,1
401030714,[],2018,1
401030713,[],2018,1
401030712,[],2018,1
401030711,[],2018,1


In [53]:
# Update plan
# Get week's new gameids
# Scrape pages for new games, saving progress over existing json files
def update_json(filename, df):
    # Check for existing file
    if os.path.isfile(filename):
        read_df = pd.read_json(filename, convert_axes=False)
        # Merge new results over what's in the file
        for gid in df.index:
            read_df.loc[gid] = df.loc[gid,:]
    else:
        read_df = df
    # Write DataFrame to json file
    read_df.to_json(filename)

def big_update(season,week):
    
    # Get games that occur(red)
    weeks_games = read_espn.weekly_games(season,week)
    # Scrape individual pages for those games
    (games_df, drives_df, pbp_df) = scrape_gameset(weeks_games)
    
    # Update game history file
    filesdir = "/home/welced12/googledrive/nfl_data/devl/"
    games_file = filesdir+"espn_gamedata.json"
    drives_file = filesdir+"espn_drives.json"
    pbp_file = filesdir+"espn_rawplays.json"
    print(games_df.head(5))
    update_json(games_file, games_df)

    print(drives_df.head(5))
    update_json(drives_file, drives_df)
    
    print(pbp_df.head(5))
    print(pbp_df.columns)
    update_json(pbp_file, pbp_df)

In [54]:
big_update(2017,2)

400951620
400951632
400951636
400951639
400951643
400951645
400951630
400951635
400951626
400951669
400951666
400951673
400951674
400951676
400951679
400951681
                        result  season  week home away
gameid                                                
400951620        HOU 13, CIN 9    2017     2  HOU  CIN
400951632  ARI 16, IND 13 (OT)    2017     2  ARI  IND
400951636        KC 27, PHI 20    2017     2  PHI   KC
400951639         NE 36, NO 20    2017     2   NE   NO
400951643        PIT 26, MIN 9    2017     2  MIN  PIT
  away away_score_after away_score_before  drive home home_score_after  \
0  CIN                0                 0      0  HOU                0   
1  CIN                0                 0      1  HOU                0   
2  CIN                0                 0      2  HOU                0   
3  CIN                0                 0      3  HOU                0   
4  CIN                0                 0      4  HOU                0   

  home_sco

ValueError: cannot set a row with mismatched columns

In [39]:
x = big_update(2017,3)
x[(x.season==2017) & (x.week==3)]

400951568
400951579
400951583
400951590
400951594
400951599
400951604
400951607
400951611
400951616
400951587
400951623
400951627
400951634
400951644
400951668


,result,season,week
400951568,"LAR 41, SF 39",2017,3
400951579,"JAX 44, BAL 7",2017,3
400951583,"BUF 26, DEN 16",2017,3
400951587,"NO 34, CAR 13",2017,3
400951590,"CHI 23, PIT 17 (OT)",2017,3
400951594,"ATL 30, DET 26",2017,3
400951599,"IND 31, CLE 28",2017,3
400951604,"MIN 34, TB 17",2017,3
400951607,"NE 36, HOU 33",2017,3
400951611,"NYJ 20, MIA 6",2017,3


In [2]:
import time
import read_espn
import pandas as pd
import os
import sys


def update_json(filename, df):
    # Check for existing json file consisting of a single DataFrame
    if os.path.isfile(filename):
        read_df = pd.read_json(filename, convert_axes=False)
        # Write new results on top of those from file
        for gid in df.index:
            read_df.loc[gid] = df.loc[gid,:]
    else:
        read_df = df

    # Write DataFrame to json file
    read_df.to_json(filename)


def big_update(season, week):

    # Get games that occur(red)
    weeks_games = read_espn.weekly_games(season, week)
    # Scrape individual pages for those games
    (games_df, drives_df, pbp_df) = read_espn.scrape_gameset(weeks_games)

    # Update existing results
    filesdir = "/home/welced12/googledrive/nfl_data/devl/"
    games_file = filesdir+"espn_gamedata.json"
    drives_file = filesdir+"espn_drives.json"
    pbp_file = filesdir+"espn_rawplays.json"
    print(games_df.head(5))
    print(drives_df.head(5))
    print(pbp_df.head(5))
    update_json(games_file, games_df)
    update_json(drives_file, drives_df)
    update_json(pbp_file, pbp_df)

In [11]:
read_df = pd.read_json("/home/welced12/googledrive/nfl_data/devl/espn_drives.json",
                       convert_axes=False)

In [12]:
read_df

,away,away_score_after,away_score_before,drive,home,home_score_after,home_score_before,offense,plays,result,time,yds_gained,gameid
320905019-0,NYG,0,0,0,DAL,0,0,NYG,3,Punt,1:16,5,320905019
320905019-1,NYG,0,0,1,DAL,0,0,DAL,3,Punt,1:23,-5,320905019
320905019-10,NYG,3,3,10,DAL,7,7,NYG,4,End of Half,1:01,-1,320905019
320905019-11,NYG,3,3,12,DAL,14,7,DAL,9,Touchdown,4:36,80,320905019
320905019-12,NYG,10,3,13,DAL,14,14,NYG,9,Touchdown,5:13,77,320905019
320905019-13,NYG,10,10,14,DAL,17,14,DAL,8,Field Goal,4:31,65,320905019
320905019-14,NYG,10,10,15,DAL,17,17,NYG,5,Punt,2:37,21,320905019
320905019-15,NYG,10,10,16,DAL,24,17,DAL,8,Touchdown,7:06,92,320905019
320905019-16,NYG,17,10,17,DAL,24,24,NYG,12,Touchdown,3:21,79,320905019
320905019-17,NYG,17,17,18,DAL,24,24,DAL,8,End of Game,2:36,8,320905019


In [13]:
pbp = pd.read_json("/home/welced12/googledrive/nfl_data/devl/espn_rawplays.json")
pbp.head(5)

,downdist,detail,play_num,home,away,possession,home_score_before,away_score_before,home_score_after,away_score_after,drive_num,gameid
290910023-0,[1st & 10 at PIT 42],(14:53 - 1st) B.Roethlisberger pass short left...,1,TEN,PIT,PIT,0,0,0,0,0,290910023
290910023-1,[2nd & 5 at PIT 47],(14:16 - 1st) W.Parker right end to PIT 44 for...,2,TEN,PIT,PIT,0,0,0,0,0,290910023
290910023-10,[1st & 10 at TEN 30],(10:21 - 1st) W.Parker right guard to TEN 31 f...,3,TEN,PIT,PIT,0,0,0,0,2,290910023
290910023-100,[2nd & 9 at TEN 6],(9:11 - 3rd) C.Johnson left end to TEN 6 for n...,2,TEN,PIT,TEN,7,7,7,7,16,290910023
290910023-101,[3rd & 9 at TEN 6],(8:46 - 3rd) C.Johnson right end to TEN 4 for...,3,TEN,PIT,TEN,7,7,7,7,16,290910023


In [ ]:
def process_drives(drives_df):
    